# Configureation
## Data
### Featuring
* Features: Pclass, Sex, Ticket, Cabin, Embarked, IsAlone, Age, Fare
* **Features are normalized before process, using scaler fit from train data, and appying to test data for scale consistency**

## Evolutionary process:
### Individual
* LISP tree
* Min depth: 4
* Max depth: 8
* primitives: addsubtract, multiply, sin, cost, tan, **sigmoid**, **power of 2**, ~~power of 3~~
### Evaluation
* **0.5 threshold to round indivudal score output to 0 or 1**
* objectives: -fnr, -fpr
### Variation
#### 'varAnd' derived implementation
first implemented deap.algorithms.eaMuPlusLambda with `varOr` implementation replaced by `varAnd` implementation. then based on that, added various mutate capabilities: mutUniform, mutReplaceNode, mutShrink
#### Mate
**gp.CxOnePointLeafBiased()**
#### Mutate
**mutUniform**, **mutReplaceNode**, **mutShrink**
### Select
tools.selNSGA2
### Other
number of child: 100, geneartion number: **1500**, number of population: 50, mutate prob: 0.1, mate prob: 0.5

In [ ]:
import random
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from deap import algorithms
import seaborn as sns
from deap import base
from deap import creator
from deap import tools
from deap import gp

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

pd.options.mode.chained_assignment = None  # ignore SettingWithCopyWarning
scaler = MinMaxScaler()
np_logger = np.seterr(all='warn', over='ignore', under='ignore')  # np.power, np.exp may create underflow or overflow 

# features

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

map_cabin = dict(zip(train_data.Cabin.apply(lambda x: x[:1] if x is not np.nan else ''), 
                     train_data.Cabin.apply(lambda x: x[:1] if x is not np.nan else '').astype('category').cat.codes))
map_ticket = dict(zip(train_data.Ticket.apply(lambda x: x[:3]), 
                      train_data.Ticket.apply(lambda x: x[:3]).astype('category').cat.codes))
map_sex = {'male': 0, 'female': 1}
map_embarked = dict(zip(train_data.Embarked.fillna(''), train_data.Embarked.fillna('').astype('category').cat.codes))
labels = ["{0} - {1}".format(i, i + 19) for i in range(0, 90, 18)]
map_AgeRange = dict(zip(sorted(labels), range(len(labels))))
map_FareRange = dict(zip([0, 5, 10, 15, 20, 30], [0, 1, 2, 3, 4, 5]))

if False:  # my own features, difference from Devan's is Cabin, Ticket, and how some features are catergorized
    for data in (train_data, test_data):
        data['Cabin'] = data.Cabin.apply(lambda x: x[:1] if x is not np.nan else '').map(map_cabin)
        data['Ticket'] = data.Ticket.apply(lambda x: x[:3]).map(map_ticket).fillna(-1)
        data['Sex'] = data.Sex.map(map_sex)
        data['HasSibSp'] = data.apply(lambda x: 1 if x.SibSp > 0 else 0, axis=1)
        data['HasParCh'] = data.apply(lambda x: 1 if x.Parch > 0 else 0, axis=1)
        data['IsAlone'] = data.apply(lambda x: 0 if x.SibSp > 0 or x.Parch > 0 else 1, axis=1)
        data['Embarked'] = data.Embarked.fillna('').map(map_embarked)
        data['Fare'] = data.Fare.fillna(data.Fare.median()) # fill missing
        data['Age'] = data.Age.fillna(data.Age.median())  # fill missing
        data['Age'] = pd.cut(data.Age, range(0, 108, 18), right=False, labels=labels).map(map_AgeRange)
        data['Fare'] = pd.cut(data.Fare, 
                                   [0, 5, 10, 15, 20, 30, 1000], 
                                   right=False, 
                                   labels=[0, 5, 10, 15, 20, 30]).map(map_FareRange)


        # do features here
        del data['Name']
        data.set_index('PassengerId', inplace=True)
        # del data['PassengerId']
        del data['SibSp']
        del data['Parch']
        del data['HasSibSp']
        del data['HasParCh']
        # del data['IsAlone']
        # del data['Cabin']
        # del data['Ticket']
else:  # Devan
    for dataset in (train_data, test_data):
        # Create boolean 'IsAlone' to represent 'Sibsp' and 'Parch'
        dataset['IsAlone'] = 0
        dataset.loc[dataset['SibSp'] + dataset['Parch'] == 0, 'IsAlone'] = 1

        # Drop irrelevant/cumbersome columns and make 'PassengerId' the index
        dataset.drop(columns=['Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], inplace=True)
        dataset.set_index(keys=['PassengerId'], drop=True, inplace=True)

        # Create map and fill NaN values
        dataset_nan_map = {
            'Age': dataset['Age'].mean(),
            'Fare': dataset['Fare'].mean(),
        }
        dataset.fillna(value=dataset_nan_map, inplace=True)

        # map mixed types to numbers and fill nan values
        columns_map = {
            'Sex': {'male': 0, 'female': 1},
        }
        dataset.replace(columns_map, inplace=True)

        dataset['Embarked'] = dataset.Embarked.fillna('').map(map_embarked)

        # Change to ordinal values based on bands created above
        dataset['Age'] = dataset["Age"].apply(lambda x: 0 if x <= 16 
                                              else 1 if x > 16 and x <= 32
                                              else 2 if x > 32 and x <= 48
                                              else 3 if x > 48 and x <= 64
                                              else 4)
        dataset['Fare'] = dataset["Fare"].apply(lambda x: 0 if x <= 7.91 
                                                else 1 if x > 7.91 and x <= 14.454
                                                else 2 if x > 14.454 and x <= 31
                                                else 3)
    
X_train = train_data.loc[:, train_data.columns != 'Survived']
y_train = train_data.loc[:, 'Survived']

scaler.fit(X_train)

X_train.loc[:, :] = scaler.transform(X_train.values)
test_data.loc[:, :] = scaler.transform(test_data.values)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=10)

print(X_train.head())
print(test_data.head())

In [ ]:
# check table columns that contain missing values
for data in (train_data, test_data):
    print(data.columns[data.isna().any()].tolist())

In [ ]:
# Pearson Correlation Matrix
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_data.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

random.seed(25)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def pow2(x):
    return np.power(x, 2)

def pow3(x):
    return np.power(x, 3)

def pow4(x):
    return np.power(x, 4)

pset = gp.PrimitiveSet("MAIN", arity=len(X_train.columns))
pset.addPrimitive(np.add, arity=2)
pset.addPrimitive(np.subtract, arity=2)
pset.addPrimitive(np.multiply, arity=2)
pset.addPrimitive(np.sin, arity=1)
# pset.addPrimitive(np.cos, arity=1)
# pset.addPrimitive(np.tan, arity=1)
pset.addPrimitive(sigmoid, arity=1)
# pset.addPrimitive(pow2, arity=1)
# pset.addPrimitive(pow3, arity=1)
# pset.addPrimitive(pow4, arity=1)
# pset.addEphemeralConstant('const', lambda: random.uniform(-1, 1))

pset.renameArguments(ARG0='x')

toolbox = base.Toolbox()

toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=4, max_=8)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [ ]:
def evalFPRFNR(individual, X, y, pset):
    func = gp.compile(expr=individual, pset=pset)
    pred = func(X.Pclass.values, X.Sex.values, X.Age.values, X.Fare.values, X.Embarked.values, X.IsAlone.values)
    # pred = func(X.Pclass.values, X.Sex.values, X.Age.values, X.Fare.values, X.Embarked.values, X.IsAlone.values, 
    #             X.Cabin.values, X.Ticket.values)
    pred = np.array([0 if x < 0.5 else 1 for x in pred]).reshape(-1, 1)
    
    tn, fp, fn, tp = confusion_matrix(y.values, pred).ravel()
    fpr = fp/(fp+tn) if (fp+tn) != 0 else 1
    fnr = fn/(fn+tp) if (fn+tp) != 0 else 1
    
    return (fpr, fnr)


toolbox.register("evaluate", evalFPRFNR, X=X_train, y=y_train, pset=pset)
toolbox.register("select", tools.selNSGA2)  # best so far
# toolbox.register("select", tools.selSPEA2)  # okay, slightly worse than NSGA2
# toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mate", gp.cxOnePointLeafBiased)

toolbox.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("mutnode", gp.mutNodeReplacement, pset=pset)
toolbox.register("mutshrink", gp.mutShrink)
# toolbox.register("mutconstant", gp.mutEphemeral, mode='all')

# bloat control: DEAP trees can only have a maximum depth of 91 due to limitations in Python's parser
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
toolbox.decorate("mutnode", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
toolbox.decorate("mutshrink", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
# toolbox.decorate("mutconstant", gp.staticLimit(key=operator.attrgetter("height"), max_value=7))

In [ ]:
def pareto_dominance(ind1, ind2):
    not_equal = False
    for value_1, value_2 in zip(ind1.fitness.values, ind2.fitness.values):
        if value_1 > value_2:
            return False
        elif value_1 < value_2:
            not_equal = True
    return not_equal

# main evolutionary algorithm.

In [ ]:
NGEN = 50
NXTGEN = 50
NCHILD = 100
CXPB = 0.5
MUTPB = 0.15
CXTEMPB = 0  # 0 is best; when it is 1, cxonepointleafbaised is worse than cxonepoint


pop = toolbox.population(n=NXTGEN)
hof = tools.ParetoFront()
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

logbook = tools.Logbook()
logbook.header = ['gen'] + (stats.fields)

invalid_ind = [ind for ind in pop if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit
    
record = stats.compile(pop) 
logbook.record(gen=0, **record)
print(logbook.stream)
for gen in range(NGEN):
        # Vary the population
        offspring = []
        for x in range(NCHILD):
            # basically varOr https://deap.readthedocs.io/en/master/api/algo.html#deap.algorithms.varOr
            if False:
                choice = random.random()
                if choice < CXPB: # crossover
                    ind1, ind2 = map(toolbox.clone, random.sample(pop, 2))
                    ind1, ind2 = toolbox.mate(ind1, ind2)
                    del ind1.fitness.values
                    offspring.append(ind1)
                elif choice < CXPB + MUTPB: # mutation
                    ind = toolbox.clone(random.choice(pop))
                    ind, = toolbox.mutate(ind)
                    del ind.fitness.values
                    offspring.append(ind)
                else: # reproduction
                    offspring.append(random.choice(pop))
            # changed a lot from varAnd https://deap.readthedocs.io/en/master/api/algo.html#deap.algorithms.varAnd
            else:
                offspring = list(map(toolbox.clone, pop))
                for child1, child2 in zip(offspring[::2], offspring[1::2]): # mate
                    if random.random() < CXPB:
                        toolbox.mate(child1, child2, CXTEMPB)
                        del child1.fitness.values
                        del child2.fitness.values
                for mutant in offspring: # mutate
                    # if random.random() < MUTPB:
                    #     toolbox.mutate(mutant)
                    #     del mutant.fitness.values
                    if random.random() < MUTPB:
                        toolbox.mutnode(mutant)
                        del mutant.fitness.values
                    # if random.random() < MUTPB:
                    #     toolbox.mutconstant(mutant)
                    #     del mutant.fitness.values
                    # if random.random() < MUTPB:
                    #     toolbox.mutshrink(mutant)
                    #     del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if hof is not None:
            hof.update(offspring)

        # Select the next generation population
        pop[:] = toolbox.select(pop + offspring, NXTGEN)

        # Update the statistics with the new population
        record = stats.compile(pop)
        logbook.record(gen=gen, **record)
        print(logbook.stream)

Now, we will plot the results of our run and display the best individual.

In [ ]:
print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))
gen, avg, min_, max_, std = logbook.select("gen", "avg", "min", "max", "std")
plt.plot(gen, avg, label="average")
plt.plot(gen, min_, label="minimum")
plt.plot(gen, std, label="std")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="upper left")
plt.rcParams["figure.figsize"] = (20, 10)

plt.show()

In [ ]:
# performance on train dataset

"""Split fitness values into separate lists"""
fitness_1 = [ind.fitness.values[0] for ind in hof]
fitness_2 = [ind.fitness.values[1] for ind in hof]
pop_1 = [ind.fitness.values[0] for ind in pop]
pop_2 = [ind.fitness.values[1] for ind in pop]

'''Print dominated population for debugging'''
# for ind in pop:
#     print(ind.fitness)

plt.scatter(pop_1, pop_2, color='b')
plt.scatter(fitness_1, fitness_2, color='r')
plt.plot(fitness_1, fitness_2, color='r', drawstyle='steps-post')
plt.xlabel("FPR")
plt.ylabel("FNR")
plt.title("Pareto Front")
plt.show()

f1 = np.array(fitness_1)
f2 = np.array(fitness_2)

"""Calculate area under curve with least squares method"""
print("Area Under Curve: %s" % (np.sum(np.abs(np.diff(f1))*(np.array(fitness_2)[:-1] + np.array(fitness_2)[1:])/2)))
print(f'hof num {len(hof)}')

In [ ]:
# predict on test dataset
pred_df = pd.DataFrame(index=test_data.index)

for i, ind in enumerate(hof):
    func = gp.compile(expr=ind, pset=pset)
    pred = func(test_data.Pclass.values, test_data.Sex.values, test_data.Age.values, test_data.Fare.values, 
                test_data.Embarked.values, test_data.IsAlone.values)
    # pred = func(test_data.Pclass.values, test_data.Sex.values, test_data.Age.values, test_data.Fare.values, 
    #             test_data.Embarked.values, test_data.IsAlone.values, test_data.Cabin.values, test_data.Ticket.values)
    pred = np.array([0 if x < 0.5 else 1 for x in pred])
    pred_df["HOF %i" % i] = pred
print(pred_df.head(3))

# pred_df.to_csv('predictions.csv', header=True, sep=',')

# GP vs ML

In [ ]:
GP_on_full_train_data = False

if not GP_on_full_train_data:
    # _, X_test, _, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=10)
    toolbox.register("evaluate_validate", evalFPRFNR, X=X_test, y=y_test, pset=pset)
    # re-evaluate GP hof on validation data to compare to ML hof
    for ind in hof:
        del ind.fitness.values
    fitnesses = toolbox.map(toolbox.evaluate_validate, hof)
    for ind, fit in zip(hof, fitnesses):
        ind.fitness.values = fit
    hof_valid = tools.ParetoFront()
    hof_valid.update(hof)  # sorted on 1st score ascending

In [ ]:
# performance on validation dataset

def _calculate_AUC(ML_0, ML_1, hof_valid):
    """
    Calculate area under curve to compare ML and GP solutions
    
    ML individuals are fewer. So cut min and max X-axis score from ML individuals as the border to calcualte area
    For both ML and GP individuals. 
    
    """
    cut0_x, cut1_x = ML_1[0], ML_1[-1]
    
    hof_valid_set = set()
    for ind in hof_valid:
        hof_valid_set.add(ind.fitness.values)
    hof_valid_set = sorted(list(hof_valid_set))
    
    _x = next(i for (i, v) in enumerate(hof_valid_set) if v[0] >= cut0_x)  # first index in hof where first score above left cut
    _w = (cut0_x - hof_valid_set[_x-1][0]) / (hof_valid_set[_x][0] - hof_valid_set[_x-1][0])  # corresponding weight
    cut0_y = hof_valid_set[_x-1][1]*(1-_w) + hof_valid_set[_x][1]*(_w)
    _x, _w, cut0_y
    
    _x = next(i for (i, v) in enumerate(hof_valid_set) if v[0] >= cut1_x)  # first index in hof where first score above right cut
    _w = (cut1_x - hof_valid_set[_x-1][0]) / (hof_valid_set[_x][0] - hof_valid_set[_x-1][0])  # corresponding weight
    cut1_y = hof_valid_set[_x-1][1]*(1-_w) + hof_valid_set[_x][1]*(_w)
    _x, _w, cut1_y
    
    hof_4auc = [x for x in hof_valid_set if x[0]>= cut0_x and x[0]<= cut1_x]
    hof_4auc.append((cut0_x, cut0_y))
    hof_4auc.append((cut1_x, cut1_y))
    hof_4auc = sorted(hof_4auc)
    
    print("AUC GP: %.4f" % (np.sum(np.abs(np.diff(np.array([x[0] for x in hof_4auc])))*(np.array([x[0] for x in hof_4auc])[:-1] + np.array([x[1] for x in hof_4auc])[1:])/2)))
    print("AUC ML: %.4f" % (np.sum(np.abs(np.diff(np.array(ML_1)))*(np.array(ML_2)[:-1] + np.array(ML_2)[1:])/2)))
    print(f'GP HOF# {len(hof_valid)}  (include individuals with same scores)')
    print(f'ML HOF# {len(names)}')
    
    return

    
ML_1 = [0.048, 0.0612, 0.0680, 0.1156, 0.2300]  # sorted on 1st score ascending
ML_2 = [0.4342, 0.4079, 0.3684, 0.3553, 0.1600]
names = ['David', 'Zhao', 'Dhruv', 'Devesh', 'Devan']
fitness_1 = [ind.fitness.values[0] for ind in hof_valid]
fitness_2 = [ind.fitness.values[1] for ind in hof_valid]
plt.plot(ML_1, ML_2, color='b', linestyle='solid', marker='o')
for i in range(5):
    plt.annotate(text=names[i], xy=(ML_1[i], ML_2[i]))
plt.axvline(x=ML_1[0], linestyle=':')
plt.axvline(x=ML_1[-1], linestyle=':')
plt.scatter(fitness_1, fitness_2, color='r')
plt.plot(fitness_1, fitness_2, color='r', drawstyle='steps-post')
plt.xlabel("FPR")
plt.ylabel("FNR")
plt.title("Pareto Front ML vs GP on same unseen validation dataset")
plt.show()

_calculate_AUC(ML_1, ML_2, hof_valid)